## NLP 
I use AWS to performe my NLP processing

In [2]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO

In [55]:
sys.path.insert(1, '../src')
from NLP import nlp_process
from Content_recommender import ContentRecommender

In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
%reload_ext autoreload

In [6]:
s3 = boto3.client('s3')

In [7]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv")

In [8]:
df_all_15.shape

(370252, 13)

In [9]:
df_all_15.head(5)

,Unnamed: 0,category,title,also_buy,image_x,rank,also_view,price,asin,overall,verified,text,title_nlp
0,0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",$19.95,0899332757,5.0,False,garminif your interest explor area new york be...,delorm new york state atla amp gazett
1,1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",$19.95,0899332757,4.0,True,garminwond comprehens atla littl largefour sta...,delorm new york state atla amp gazett
2,2,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",$19.95,0899332757,5.0,True,garmingreat product fast deliveri need rv trav...,delorm new york state atla amp gazett
3,3,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,"['>#84,484 in Office Products (See top 100)', ...",[],$21.96,0899333257,5.0,True,garmini delorm atla us state theyr amaz detail...,garmin delorm atla amp gazett paper map arizona
4,4,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,"['>#84,484 in Office Products (See top 100)', ...",[],$21.96,0899333257,5.0,False,garminth first delorm atla gazett ever purchas...,garmin delorm atla amp gazett paper map arizona


In [12]:
df_all_15 = df_all_15[~(df_all_15['title_nlp'].isna())]
df_all_15 = df_all_15[~(df_all_15['text'].isna())]
                         

In [10]:
df_all_15.shape

(370252, 13)

In [35]:
df_items = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['text'].apply(lambda x: ' '.join(x)).reset_index()

In [36]:
df_items

,asin,title,title_nlp,image_x,text
0,0899332757,Delorme New York State Atlas &amp; Gazetteer,delorm new york state atla amp gazett,['https://images-na.ssl-images-amazon.com/imag...,garminif your interest explor area new york be...
1,0899333257,Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,garmin delorm atla amp gazett paper map arizona,['https://images-na.ssl-images-amazon.com/imag...,garmini delorm atla us state theyr amaz detail...
2,0971100764,BenchMaster Pocket Guide - Fly Fishing - Fishing,benchmast pocket guid fli fish fish,['https://images-na.ssl-images-amazon.com/imag...,pocket guidesmi husband know everyth tie knot ...
3,3843518912,Spooner Boards Freestyle - Green,spooner board freestyl green,['https://images-na.ssl-images-amazon.com/imag...,spooner boardsw purchas year old son love play...
4,7245456275,Black Mountain Products Single Resistance Band...,black mountain product singl resist band door ...,['https://images-na.ssl-images-amazon.com/imag...,black mountaingreat band price door attach wor...
...,...,...,...,...,...
56393,B01HJA7KWE,MEILAIER UV Fishing Mask Moisture Wicking Head...,meilaier uv fish mask moistur wick headband se...,['https://images-na.ssl-images-amazon.com/imag...,note unit suppli reviewmeilai suppli headband ...
56394,B01HJDGJ1E,"Soft Cooling Towel for Instant Relief, 48inch ...",soft cool towel instant relief extra long natu...,['https://images-na.ssl-images-amazon.com/imag...,pusdonif choic type cool towel dri rock hard c...
56395,B01HJDHNX2,Cycling Computers- Wireless Bike Speedometer a...,cycl comput wireless bike speedomet odometerwa...,['https://images-na.ssl-images-amazon.com/imag...,perfect speedomet anyon look larg display inex...
56396,B01HJDZ34I,Columbia Bora booney,columbia bora booney,['https://images-na.ssl-images-amazon.com/imag...,columbiadec hat work well normal duti need sti...


In [37]:
s3.list_buckets()['Buckets']

[{'Name': 'recommender-system-amazon',
  'CreationDate': datetime.datetime(2020, 9, 14, 18, 7, 18, tzinfo=tzlocal())}]

In [38]:
bucket = 'recommender-system-amazon' 
csv_buffer = StringIO()
df_items.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df_items.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '0C83083E76EF49D8',
  'HostId': 'x/j+aGQNfICqUYwPGShpAKdZvLpIfAFCUg5FCE/5NIja1rsIlUptr2aUS8AivbRftC8yloAo05o=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'x/j+aGQNfICqUYwPGShpAKdZvLpIfAFCUg5FCE/5NIja1rsIlUptr2aUS8AivbRftC8yloAo05o=',
   'x-amz-request-id': '0C83083E76EF49D8',
   'date': 'Tue, 15 Sep 2020 22:18:35 GMT',
   'etag': '"e55615f1ef880c528957b4fb8bf75c1f"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"e55615f1ef880c528957b4fb8bf75c1f"'}

## Process text and title_nlp separately

In [39]:
vectorizer_text, df_text = nlp_process(df_items['text'],max_features = None)

In [40]:
df_text

,aa,aaa,ab,abil,abl,absolut,absorb,abus,accept,access,...,young,younger,youth,youtub,youv,yr,zero,zip,zipper,zone
0,0.0,0.0,0.0,0.0,0.090639,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.094692,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.113687,0.000000,0.0,0.108667,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.013326,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56393,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
56394,0.0,0.0,0.0,0.0,0.043837,0.027770,0.0,0.0,0.0,0.0,...,0.000000,0.040593,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
56395,0.0,0.0,0.0,0.0,0.069576,0.044076,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.052592,0.0,0.0
56396,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [41]:
vectorizer_title, df_title = nlp_process(df_items['title_nlp'],max_features = None)

In [42]:
df_title

,accessori,activ,adida,adjust,adult,air,airsoft,aluminum,amp,arm,...,wide,winter,women,wood,workout,wrap,wrist,wristband,yoga,youth
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.588029,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0
56394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.143763,0.0,...,0.0,0.0,0.0,0.0,0.195138,0.230085,0.0,0.0,0.18775,0.0
56395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0
56396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0


In [43]:
df_text.to_csv(csv_buffer)
s3_resource.Object(bucket, 'df_text.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '5AC3EF2111EFC682',
  'HostId': 'Etxo8aEwhTbv3RtEoAdmgHuvkGoicyD0sauY9JB1+Qplz43W1uPYmmPrHfZv886kyqS8BGGNiIA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Etxo8aEwhTbv3RtEoAdmgHuvkGoicyD0sauY9JB1+Qplz43W1uPYmmPrHfZv886kyqS8BGGNiIA=',
   'x-amz-request-id': '5AC3EF2111EFC682',
   'date': 'Tue, 15 Sep 2020 22:23:02 GMT',
   'etag': '"543ed8010cef492b570033d276c42157"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"543ed8010cef492b570033d276c42157"'}

In [44]:
df_title.to_csv(csv_buffer)
s3_resource.Object(bucket, 'df_title.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '603B2578A56B7906',
  'HostId': 'M8ivFyAOgn4CGKj96f/Yd0FArJO+2xZzmjqOF/8NHgNrWqLlFYdzqfBBi027vY98dM/lEyvq658=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'M8ivFyAOgn4CGKj96f/Yd0FArJO+2xZzmjqOF/8NHgNrWqLlFYdzqfBBi027vY98dM/lEyvq658=',
   'x-amz-request-id': '603B2578A56B7906',
   'date': 'Tue, 15 Sep 2020 22:23:32 GMT',
   'etag': '"a21e770cfc2d06614b0058846c8f82f6"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"a21e770cfc2d06614b0058846c8f82f6"'}

In [45]:
df_items['title']

0             Delorme New York State Atlas &amp; Gazetteer
1        Garmin DeLorme Atlas &amp; Gazetteer Paper Map...
2         BenchMaster Pocket Guide - Fly Fishing - Fishing
3                         Spooner Boards Freestyle - Green
4        Black Mountain Products Single Resistance Band...
                               ...                        
56393    MEILAIER UV Fishing Mask Moisture Wicking Head...
56394    Soft Cooling Towel for Instant Relief, 48inch ...
56395    Cycling Computers- Wireless Bike Speedometer a...
56396                                 Columbia Bora booney
56397                    2XU Men's Core Compression Shorts
Name: title, Length: 56398, dtype: object

In [56]:
news_recommender = ContentRecommender()
news_recommender.fit(df_title, titles = df_items['title'])

In [ ]:
news_recommender.get_recommendations()